In [1]:
import pandas as pd
import sklearn
from matplotlib import pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LassoCV

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.metrics import multilabel_confusion_matrix

%matplotlib inline

import statsmodels.api as sm
from statsmodels.api import OLS

In [2]:
crime_property_light_population= pd.read_csv("./model_data/crime_light_density.csv")
crime_property_light_population.head()

predictors = ['OFFENSE_CODE_GROUP','SHOOTING', 'MONTH', 'DAY_OF_WEEK', 'HOUR', 'Population density (per square mile of land area)', 'BLDG_VAL', 'LAND_VAL','light_density']
df = crime_property_light_population[predictors]
print(len(df))

396321


In [3]:
### USE THIS CODE TO DROPNA###
df = df.dropna()
print(len(df))
df.head()

387177


,OFFENSE_CODE_GROUP,SHOOTING,MONTH,DAY_OF_WEEK,HOUR,Population density (per square mile of land area),BLDG_VAL,LAND_VAL,light_density
0,Auto Theft,N,10,Sunday,9,6206.9,455.644447,21.073507,0.437849
1,Auto Theft,N,10,Saturday,20,6206.9,455.644447,21.073507,0.437849
2,Auto Theft,N,10,Saturday,15,16212.0,2250.588503,63.554783,0.582893
3,Auto Theft,N,10,Saturday,4,32724.1,14799.401260,480.463801,0.656585
4,Auto Theft,N,10,Wednesday,8,12192.0,278.377510,165.249738,0.678941


In [4]:
####USE THIS CODE TO FILLNA##
# df = df.fillna(method='ffill')
# print(len(df))
# df.head()

In [5]:
top5 = df['OFFENSE_CODE_GROUP'].value_counts().index[:5]

#cut the df
df = df[df['OFFENSE_CODE_GROUP'].isin(top5)]

print(len(df))

147858


In [6]:
#Let's label encode & one-hot encode the Categoricals (SHOOTING, DAY_OF_WEEK)

enc = OneHotEncoder(handle_unknown='ignore')

le = LabelEncoder() 
  
df['SHOOTING']= le.fit_transform(df['SHOOTING'])
df['DAY_OF_WEEK']= le.fit_transform(df['DAY_OF_WEEK']) 
df['OFFENSE_CODE_GROUP'] = le.fit_transform(df['OFFENSE_CODE_GROUP'])
print(len(df))

147858


In [7]:
df_cat = enc.fit_transform(df[["MONTH","SHOOTING","DAY_OF_WEEK", "HOUR"]]).toarray()
df_cat = pd.DataFrame(df_cat, columns=enc.get_feature_names(['MONTH','SHOOTING', 'DAY_OF_WEEK', 'HOUR']))
display(df_cat.head())

print(len(df_cat))

,MONTH_1,MONTH_2,MONTH_3,MONTH_4,MONTH_5,MONTH_6,MONTH_7,MONTH_8,MONTH_9,MONTH_10,...,HOUR_14,HOUR_15,HOUR_16,HOUR_17,HOUR_18,HOUR_19,HOUR_20,HOUR_21,HOUR_22,HOUR_23
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


147858


In [8]:
# display(df)
df.reset_index(drop=True, inplace=True)
df_cat.reset_index(drop=True, inplace=True)
df = pd.concat([df, df_cat], axis=1)
# df = df.dropna()
print(len(df))
df.head()


147858


,OFFENSE_CODE_GROUP,SHOOTING,MONTH,DAY_OF_WEEK,HOUR,Population density (per square mile of land area),BLDG_VAL,LAND_VAL,light_density,MONTH_1,...,HOUR_14,HOUR_15,HOUR_16,HOUR_17,HOUR_18,HOUR_19,HOUR_20,HOUR_21,HOUR_22,HOUR_23
0,0,0,9,3,3,26693.9,3362.723369,581.882742,0.699176,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3,0,9,3,5,24074.6,5490.169358,260.905478,0.728900,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,9,3,1,8352.4,2009.317606,59.185114,0.525616,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0,9,3,0,15913.2,1293.230035,31.015855,0.568578,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,9,3,3,14884.4,1040.984096,31.260815,0.640733,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
categ_to_drop = ['SHOOTING', 'SHOOTING_0', 'MONTH_1', 'DAY_OF_WEEK_0', 'DAY_OF_WEEK', 'MONTH', 'HOUR', 'HOUR_0']

# DROP SHOOTING 
# DROP SHOOTING_0
# DROP HOUR_0
# DROP HOUR
# DROP MONTH
# DROP MONTH1
# DROP DAY_OF_WEEK_0
# DROP DAY_OF_WEEK
df = df.drop(categ_to_drop, axis=1)
# add zipcode as encoded categorical  # 


df.head()

,OFFENSE_CODE_GROUP,Population density (per square mile of land area),BLDG_VAL,LAND_VAL,light_density,MONTH_2,MONTH_3,MONTH_4,MONTH_5,MONTH_6,...,HOUR_14,HOUR_15,HOUR_16,HOUR_17,HOUR_18,HOUR_19,HOUR_20,HOUR_21,HOUR_22,HOUR_23
0,0,26693.9,3362.723369,581.882742,0.699176,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3,24074.6,5490.169358,260.905478,0.728900,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,8352.4,2009.317606,59.185114,0.525616,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,15913.2,1293.230035,31.015855,0.568578,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,14884.4,1040.984096,31.260815,0.640733,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:

X_train, X_test, y_train, y_test = train_test_split(df.loc[:, df.columns != 'OFFENSE_CODE_GROUP'], 
                                                         df.OFFENSE_CODE_GROUP, test_size=0.2, 
                                                         random_state = 109, 
                                                         stratify = df.OFFENSE_CODE_GROUP)

In [11]:

X_train.head()

,Population density (per square mile of land area),BLDG_VAL,LAND_VAL,light_density,MONTH_2,MONTH_3,MONTH_4,MONTH_5,MONTH_6,MONTH_7,...,HOUR_14,HOUR_15,HOUR_16,HOUR_17,HOUR_18,HOUR_19,HOUR_20,HOUR_21,HOUR_22,HOUR_23
92390,11504.8,271.853366,28.739445,0.529698,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
64652,11504.8,271.853366,28.739445,0.529698,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
88293,24456.0,3738.859654,105.378164,0.783085,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12969,24456.0,3738.859654,105.378164,0.783085,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58611,14884.4,1040.984096,31.260815,0.640733,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
#Let's scale the variables

#Note that it is okay to leave the one-hot encoded columns for in the dataframe because min =0 max=1 anyway!
#(This won't change their values)
min_max_scaler = MinMaxScaler()
numerical = ["BLDG_VAL", "LAND_VAL", "light_density", "MONTH", "Population density (per square mile of land area)" ]

min_max_scaler.fit(X_train)
X_normalized = pd.DataFrame(min_max_scaler.transform(X_train))
X_normalized.columns = X_train.columns

X_normalized_tst = pd.DataFrame(min_max_scaler.transform(X_test))
X_normalized_tst.columns = X_test.columns


In [13]:

display(X_normalized.head())
display(X_normalized_tst.head())

,Population density (per square mile of land area),BLDG_VAL,LAND_VAL,light_density,MONTH_2,MONTH_3,MONTH_4,MONTH_5,MONTH_6,MONTH_7,...,HOUR_14,HOUR_15,HOUR_16,HOUR_17,HOUR_18,HOUR_19,HOUR_20,HOUR_21,HOUR_22,HOUR_23
0,0.139273,0.001371,0.010112,0.352743,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.139273,0.001371,0.010112,0.352743,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.334295,0.028569,0.111210,0.701468,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.334295,0.028569,0.111210,0.701468,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.190164,0.007405,0.013439,0.505555,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,Population density (per square mile of land area),BLDG_VAL,LAND_VAL,light_density,MONTH_2,MONTH_3,MONTH_4,MONTH_5,MONTH_6,MONTH_7,...,HOUR_14,HOUR_15,HOUR_16,HOUR_17,HOUR_18,HOUR_19,HOUR_20,HOUR_21,HOUR_22,HOUR_23
0,0.205025,0.031846,0.020764,0.566148,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.051409,0.000000,0.009073,0.293094,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.266095,0.003635,0.221959,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.139273,0.001371,0.010112,0.352743,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.205656,0.009384,0.013115,0.406251,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


# TRAINING MODEL STARTS FROM HERE.

In [14]:
# Let's first start with Logistic Model

print(len(X_normalized))


logreg = LogisticRegression(random_state = 0, solver='lbfgs', multi_class='multinomial', max_iter = 1000)
logreg.fit(X_normalized, y_train)
y_pred = logreg.predict(X_normalized_tst)

118286


In [15]:
mcm = multilabel_confusion_matrix(y_test, y_pred, sample_weight=None, labels=None, samplewise=False)
logreg.score(X_normalized_tst , y_test)

0.3260854862707967

In [16]:
knn = KNeighborsClassifier(n_neighbors = 15).fit(X_normalized, y_train) 
  
# accuracy on X_test 
accuracy = knn.score(X_normalized_tst, y_test) 

In [17]:
print("knn accuracy:", accuracy)
from sklearn.metrics import confusion_matrix 
cm = confusion_matrix(y_test, y_pred)

print(cm)

knn accuracy: 0.2773569592858109
[[   0 1115  234 3286    0]
 [   0 3272  207 3094    0]
 [   0 1341  465 4239    0]
 [   0 1641  354 5906    0]
 [   0 1193  231 2994    0]]


In [18]:
import pickle

pickle.dump(logreg, open('./trained_models/logreg.sav', 'wb'))
pickle.dump(knn, open('./trained_models/knn.sav', 'wb'))